In [1]:
import pandas as pd


In [2]:
DATA_PATH = "../../../data/analysis_results/gpt_4_node_retrieval_rag_based_drug_reporposing_questions.csv"


In [6]:
data = pd.read_csv(DATA_PATH)


In [27]:
import json
print(json.loads(data.llm_answer.values[-4]))

print(data.text.values[-4])
data.disease_groundTruth.values[-4]


{'Compounds': ['Uracil', 'Lenvatinib', 'TORIPALIMAB', 'Fluorouracil', 'Thiotepa', 'Alvimopan', 'Docetaxel', 'Eflornithine', 'Clavulanic Acid', 'Amoxicillin', 'Nicotinamide', 'DURVALUMAB', '1,1-Cyclobutanedicarboxylic acid', 'azane;dichloroplatinum', 'Vinblastine', 'Hexaminolevulinate', 'FILGRASTIM', 'Fenretinide', 'Valrubicin'], 'Diseases': []}
What compounds treat 'urinary bladder cancer' and what diseases resemble 'urinary bladder cancer'?


"['transitional cell carcinoma']"

In [26]:
data

,disease,compound_groundTruth,disease_groundTruth,text,llm_answer
0,Alagille syndrome,"['Odevixibat', 'Maralixibat']","['optic disk drusen', 'intrahepatic cholestasi...",What compounds treat 'Alagille syndrome' and w...,"{""Compounds"":[""Maralixibat"", ""Odevixibat""], ""D..."
1,Bardet-Biedl syndrome,['Setmelanotide'],"['retinitis pigmentosa-deafness syndrome', 'si...",What compounds treat 'Bardet-Biedl syndrome' a...,"{""Compounds"":[], ""Diseases"":[""Bardet-Biedl syn..."
2,Ehlers-Danlos syndrome,"['Celiprolol', 'Irbesartan', 'Enzastaurin']","['Loeys-Dietz syndrome', 'mitral valve prolaps...",What compounds treat 'Ehlers-Danlos syndrome' ...,"{""Compounds"":[""Celiprolol"", ""Irbesartan"", ""Enz..."
3,Ewing sarcoma,"['Treosulfan', 'Busulfan', 'Medphalan', 'Melph...","['cerebral primitive neuroectodermal tumor', '...",What compounds treat 'Ewing sarcoma' and what ...,"{""Compounds"":[""Busulfan"", ""Treosulfan"", ""Modaf..."
4,Fabry disease,"['AGALSIDASE BETA', 'Migalastat', 'AGALSIDASE ...","['familial hypertrophic cardiomyopathy', 'anhi...",What compounds treat 'Fabry disease' and what ...,"{""Compounds"":[""Lucerastat"", ""Migalastat"", ""AGA..."
...,...,...,...,...,...
138,urea cycle disorder,"['4-Phenylbutyric acid', 'Glycerol phenylbutyr...","['autosomal dominant osteopetrosis 2', 'hypera...",What compounds treat 'urea cycle disorder' and...,"{""Compounds"":[""Phenylacetic Acid"", ""4-Phenylbu..."
139,urinary bladder cancer,"['Valrubicin', 'Vinblastine', 'Lenvatinib', 'S...",['transitional cell carcinoma'],What compounds treat 'urinary bladder cancer' ...,"{""Compounds"":[""Uracil"", ""Lenvatinib"", ""TORIPAL..."
140,vascular disease,"['Clopidogrel', 'Oxygen', 'Aspirin', 'Ambrisen...","['aortic aneurysm', 'anterior spinal artery sy...",What compounds treat 'vascular disease' and wh...,"{""Compounds"":[""Oxygen"", ""Aspirin"", ""Clopidogre..."
141,viral infectious disease,"['Abacavir', 'Podofilox', 'Amenamevir', 'Falda...","['tick-borne encephalitis', 'Crimean-Congo hem...",What compounds treat 'viral infectious disease...,"{""Compounds"":[""Indinavir"", ""Letermovir"", ""Nelf..."
